Ceci est un bloc note interractif. Vous pouvez taper du code dans les cellues qui commencent par `In [ ]:` et exécuter ce code en tapant **`<Maj+Entree>`** ou en appuyant sur le bouton <button class="btn btn-default" title="Run"><i class="fa-step-forward fa"></i><span class="toolbar-btn-label">Exécuter</span></button> dans la barre ci-dessus.

Vérifier bien que vous avez complété toutes les zones contenant `TAPEZ VOTRE CODE ICI` ou "VOTRE REPONSE", ainsi que votre nom et les éventuels membres du groupe ci-dessous:

In [ ]:
NOM = ""
COLLABORATEURS = ""

---

<h1 style="text-align:center">Algorithme des $k$ plus proches voisins</h1>

<h2>Introduction</h2>

L’algorithme des $k$ plus proches voisins ($k$ nearest neightboors : knn) appartient à la famille des algorithmes d'apprentissage automatique (machine learning). L'idée de ces algorithmes est d'utiliser un grand nombre de données afin d'&laquo;apprendre&raquo; à la machine à résoudre un certain type de problème. Cette idée d'apprentissage automatique ne date pas d'hier, puisque le terme de machine learning a été utilisé pour la première fois par l'informaticien américain Arthur Samuel en 1959. Les algorithmes d'apprentissage automatique ont connu un fort regain d'intérêt au début des années 2000 notamment grâce à la quantité de données disponibles sur internet (on parle de *big data*). 

De nombreuses sociétés (exemple les GAFAM) utilisent les données concernant leurs utilisateurs afin de "nourrir" des algorithmes de machine learning qui permettront à ces sociétés d'en savoir toujours plus sur nous et ainsi de mieux cerner nos "besoins" en termes de consommation.

D'un point de vue théorique, l'algorithme des $k$ plus proches voisins est un algorithme d'apprentissage supervisé : à partir d'un ensemble E de données labellisées, il sera possible de classer (déterminer le label) d'une nouvelle donnée (donnée n'appartenant pas à E).

<h2>Présentation de l'exemple</h2>

L'exemple classique que nous allons traiter est celui des données "iris de Fisher". En 1936, Edgar Anderson a collecté des données sur 3 espèces d'iris : "iris setosa", "iris virginica" et "iris versicolor".

<table>
  <tr>
    <td style="text-align:center"><img alt="iris_setosa" src="iris_setosa.jpeg" style="width:200px; height:200px"></td>
    <td style="text-align:center"><img alt="iris_versicolor" src="iris_versicolor.jpeg" style="width:200px; height:200px"></td>
    <td style="text-align:center"><img alt="iris_virginica" src="iris_virginica.jpeg" style="width:200px; height:200px"></td>
  </tr>
  <tr>
    <td style="text-align:center">iris_setosa</td>
    <td style="text-align:center">iris_versicolor</td>
    <td style="text-align:center">iris_virginica</td>
  </tr>
<table>

Pour chaque iris étudié, Anderson a mesuré (en cm) :
   * la largeur des sépales
   * la longueur des sépales
   * la largeur des pétales
   * la longueur des pétales

Par souci de simplification, nous nous intéresserons uniquement à la largeur et à la longueur des pétales.

Pour chaque iris mesuré, Anderson a aussi noté l'espèce ("iris setosa", "iris versicolor" ou "iris virginica")

Vous trouverez 150 de ces mesures dans le fichier <a href="iris.csv">iris.csv</a>

En résumé, vous trouverez dans ce fichier :
   * la longueur des pétales
   * la largeur des pétales
   * l'espèce de l'iris (au lieu d'utiliser les noms des espèces, on utilisera des chiffres : 0 pour "iris setosa", 1 pour "iris versicolor" et 2 pour "iris virginica")

>Pour exécuter une cellule, placer le curseur dans la cellule et appuyer sur le bouton <button class="btn btn-default" title="Run"><i class="fa-step-forward fa"></i><span class="toolbar-btn-label">Run</span></button> en haut de l'écran ou sur les touches **`<Maj+Entree>`**.

><h3 class='fa fa-cogs' style="color: darkorange"> A faire vous-même </h3>
>
>Exécuter la cellule ci dessous pour avoir un aperçu du contenu du fichier <a href="iris.csv">iris.csv</a>. Les plus attentifs d'entre vous doivent reconnaître une instruction bash.

In [ ]:
cat iris.csv

<h2>Visualisation des données</h2>

Avant d'entrer dans le vif du sujet (algorithme knn), nous allons chercher à obtenir une représentation graphique des données contenues dans le fichier <a href="iris.csv">iris.csv</a>.

><h3 class='fa fa-cogs' style="color: darkorange"> A faire vous-même </h3>
>
>Étudier puis tester (en l'exécutant comme vu plus haut) le code suivant :

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def lecture(name):
    """lit le fichier csv dont le nom est passé en paramètre, en renvoie trois tableau contenant 
    respectivement la longueur et la largeur des pétales, et l'espèce de l'iris."""
    lst_long = []
    lst_larg = []
    lst_esp  = []
    with open(name) as fic:
        fic.readline()  # on ignore la première ligne contenant les titres
        for ligne in fic:
            long, larg, esp = ligne.split(',')
            lst_long.append(float(long))
            lst_larg.append(float(larg))
            lst_esp.append(int(esp))

    # conversion en tableaux numpy plus pratiques à manipuler
    return np.array(lst_long), np.array(lst_larg), np.array(lst_esp)

def affiche(long, larg, esp):
    """Représente graphiquement les données."""
    plt.xlabel("longueur")
    plt.ylabel("largeur")
    plt.title("Classification des iris")
    plt.scatter(long[esp==0], larg[esp==0], color='g', label='setosa')
    plt.scatter(long[esp==1], larg[esp==1], color='r', label='versicolor')
    plt.scatter(long[esp==2], larg[esp==2], color='b', label='virginica')
    plt.legend(loc='best')

long, larg, esp = lecture("iris.csv")
affiche(long, larg, esp)

On remarque qu'on a 3 "nuages" de points plus ou moins regroupés par espèce (même si les "iris versicolor" et les "iris virginica" se mélangent un peu).

<h2>La problématique</h2>

Imaginez maintenant qu'au cours d'une promenade vous trouviez un iris. N'étant pas un spécialiste, il ne vous est pas vraiment possible de déterminer l'espèce. En revanche, vous êtes capables de mesurer la longueur et la largeur des pétales de cet iris. Imaginons qu'un pétale fasse 2 cm de long et 0,5 cm de large. Ajoutons le point correpondant sur le graphique :

In [ ]:
plt.scatter(2, 0.5, color='k')
affiche(long, larg, esp)

Quelle est à votre avis l'espèce de cet iris ?

VOTRE REPONSE

Supposons maintenant que l'iris trouvé a des pétales de 2,4 cm de long et 0,8 cm de large. 

><h3 class='fa fa-cogs' style="color: darkorange"> A faire vous-même </h3>
>
> Taper ci-dessous les instructions pour visualiser ce cas.

In [ ]:
# TAPEZ VOTRE CODE ICI

> L'espèce paraît-elle aussi évidente ?

VOTRE REPONSE

<h2>L'agorithme des $k$ plus proches voisins</h2>

Dans ce genre de cas, il peut être intéressant d'utiliser l'algorithme des "$k$ plus proches voisins". Cet algorithme consiste à :

   * calculer la distance entre notre point (largeur du pétale = 0,8 cm ; longueur du pétale = 2,4 cm) et chaque point issu du jeu de données "iris" (à chaque fois c'est un calcul de distance entre 2 points tout ce qu'il y a de plus classique)
   * sélectionner uniquement les k distances les plus petites (les k plus proches voisins)
   * parmi les k plus proches voisins, déterminer quelle est l'espèce majoritaire. On associe à notre "iris mystère" cette "espèce majoritaire parmi les k plus proches voisins"

Prennons $k = 3$. On obtient le graphique :
<img alt="3 voisins" src="exemplek3.png">
On constate que parmi les trois plus proches voisins, 2 appartiennent à l'espèce *setosa* alors qu'un seul appartient à l'espèce *versicolor*. On peut donc supposer que l'iris trouvé appartient à l'espèce *setosa*.

<h2>Mise en œuvre</h2>

Pour pouvoir mettre l'algorithme en œuvre, on va avoir besoin de plusieurs fonctions. Tout d'abord une fonction prenant en paramètre les coordonnées de deux points et renvoyant la distance entre les deux.

Rappel : la distance entre $M(x_1,y_1)$ et $N(x_2,y_2)$ est $MN=\sqrt{(x_2-x_1)^2+(y_2-y_1)^2}$.

><h3 class='fa fa-cogs' style="color: darkorange"> A faire vous-même </h3>
>
> Compléter la fonction suivante. Pour calculer une racine carrée, vous pouvez utiliser la fonction `np.sqrt`

In [ ]:
def distance(x1, y1, x2, y2):
    """Renvoie la distance entre les points de coordonnées (x1, y1) et (x2, y2)"""
    # TAPEZ VOTRE CODE ICI

<h3>Vérifications</h3>

La cellule ci-dessous doit s'exécuter sans erreur.

In [ ]:
assert distance(1, 3, 1, 5) == 2.
assert distance(7, 1, 4, 5) == 5.
assert distance(-5, 3, 7, -2) == 13.
assert distance(42, 38, 45, 42) == 5.

Il faut alors calculer la distance entre notre point et chaque point issu du jeu de données. On trie ensuite les points par ordre croissant de distance pour ne garder que  les $k$ points les plus proches. Or, si une liste est constituée de couples, la méthode `sort` trie d'abord en fonction de la première valeur. On peut donc mémoriser des couples (distance, indice du point correspondant) dans une liste, que l'on pourra trier ensuite.

In [ ]:
def kPlusProches(x0, y0, donnees, k):
    """Renvoie la liste des indices des k points de la liste donnees les plus proches du point
    de corrdonnées (x, y).
    donnees doit être une liste de couples de coordonnées"""
    voisins = []  # la liste des indices des k voisins, avec la distance correspondante
    if k > len(donnees):  # si on demande plus de voisin qu'il n'y a de points
        k = len(donnees)  # on réduit k
    for i in range(len(donnees)):
        x, y = donnees[i]
        d = distance(x0, y0, x, y)
        voisins.append((d, i))
    voisins.sort()  # Pour un couple, sort trie d'abord en fonction de la première valeur, donc la distance
    return [voisins[i][1] for i in range(k)]  # on renvoie les indices des k points les plus proches            

Pour obtenir une liste de couples de coordonnées, on peut utiliser la fonction `zip` qui permet de passer de deux listes de valeurs à une liste de couples. La fonction renvoie cependant un objet de type `zip` qu'il faut convertir en liste pour pouvoir l'utiliser comme on veut :

In [ ]:
l1 = [1, 2, 3, 4, 5, 6]
l2 = [7, 8, 9, 10, 11, 12]
list(zip(l1, l2))

><h3 class='fa fa-cogs' style="color: darkorange"> A faire vous-même </h3>
>
> Créer deux variables `x0` et `y0` pour stocker les coordonnées (2.4, 0.8), une liste `coords` des couples de coordonnées des points du jeu de données, puis déterminer dans une variable `ind` la liste des indices des trois points les plus proches de notre point.

In [ ]:
# TAPEZ VOTRE CODE ICI

Vous pouvez tester le résultat en exécutant la cellule suivante :

In [ ]:
affiche(long, larg, esp)
plt.scatter(x0, y0, color='k')
plt.scatter(long[ind], larg[ind], color='k', marker='+')

Il ne reste plus qu'à écrire une fonction qui donne l'espèce majoritaire dans les $k$ plus proches voisins obtenus pour obtenir une classification de notre iris. Pour celà, on va avoir besoin d'une fonction qui renvoie l'**indice** du plus grand élément d'une liste.

><h3 class='fa fa-cogs' style="color: darkorange"> A faire vous-même </h3>
>
> Compléter la fonction ci-dessous

In [ ]:
def indice_max(lst):
    """Renvoie l'indice du plus grand élément de la liste non vide lst."""
    # TAPEZ VOTRE CODE ICI

<h3>Vérifications</h3>

La cellule ci-dessous doit s'exécuter sans erreur.

In [ ]:
assert indice_max([1, 0, 13, 25, 2]) == 3
assert indice_max([-1.25, 12.7, 3.5, -18.2]) == 1
assert indice_max([42, 10, 13, -4.8]) == 0
assert indice_max([3]) == 0

On peut maintenant écrire la fonction `majoritaire` :

In [ ]:
def majoritaire(ind, esp):
    """Renvoie le numéro d'espèce le plus fréquent parmi celles dont les indices sont dans ind."""
    nb_esp = [0, 0, 0]  # le nombre d'iris de chaque espèce parmi les voisins
    for i in ind:
        nb_esp[esp[i]] += 1
    return indice_max(nb_esp)

et connaître l'espèce de notre iris :

In [ ]:
majoritaire(ind, esp)

><h3 class='fa fa-cogs' style="color: darkorange"> A faire vous-même </h3>
>
> Tester d'autres coordonnées `x0` et `y0` ou d'autres valeurs de $k$ pour voir l'influence des différents paramètres.

In [ ]:
noms = ["setosa", "versicolor", "virginica"]
x0, y0 = 2.4, 0.8
k = 3
ind = kPlusProches(x0, y0, coords, k)
res = majoritaire(ind, esp)
affiche(long, larg, esp)
plt.scatter(x0, y0, color='cyan')
plt.scatter(long[ind], larg[ind], color='cyan', marker='+')
plt.text(3, 0.5, f"longueur : {x0} cm, largeur : {y0} cm")
plt.text(3, 0.3, f"Résultat pour k = {k} : {noms[res]}.")